In [67]:
import json

In [68]:
# Reading the honeypot data from the json file 
data = []
for line in open("../honeypot/log/cowrie.json", 'r'):
    data.append(json.loads(line))

In [69]:
data

[{'eventid': 'cowrie.session.connect',
  'src_ip': '172.19.0.1',
  'src_port': 47574,
  'dst_ip': '172.19.0.2',
  'dst_port': 2222,
  'session': '6bba26d07dad',
  'protocol': 'ssh',
  'message': 'New connection: 172.19.0.1:47574 (172.19.0.2:2222) [session: 6bba26d07dad]',
  'sensor': 'd4c15c73bfbf',
  'timestamp': '2023-03-02T14:28:03.451537Z'},
 {'eventid': 'cowrie.client.version',
  'version': 'SSH-2.0-OpenSSH_8.6',
  'message': 'Remote SSH version: SSH-2.0-OpenSSH_8.6',
  'sensor': 'd4c15c73bfbf',
  'timestamp': '2023-03-02T14:28:03.471436Z',
  'src_ip': '172.19.0.1',
  'session': '6bba26d07dad'},
 {'eventid': 'cowrie.client.kex',
  'hassh': 'ae8bd7dd09970555aa4c6ed22adbbf56',
  'hasshAlgorithms': 'curve25519-sha256,curve25519-sha256@libssh.org,ecdh-sha2-nistp256,ecdh-sha2-nistp384,ecdh-sha2-nistp521,diffie-hellman-group-exchange-sha256,diffie-hellman-group16-sha512,diffie-hellman-group18-sha512,diffie-hellman-group14-sha256,ext-info-c;chacha20-poly1305@openssh.com,aes128-ctr,aes192

**So what are we looking for:**
- Timestamps `timestamp`
- IPs `src_ip`
- Actions `eventid`
    - Failed login attemps `cowrie.login.failed`: `username` and `password`
    - Successful login attempts `cowrie.session.connect`: `username` and `password`
    - Inputs to the command line `input`
- Session time after eventid `cowrie.session.closed`, duration `duration`

In [78]:
login_attempts = []
ips = []
session_details = []
session_ended_details = []
commands = []
timestamps = []
login_keys_to_extract = ["username", "password", "timestamp", "src_ip", "session", "message"]
session_keys_to_extract = ['src_ip', 'src_port', 'dst_ip', 'dst_port', 'protocol', 'timestamp']
session_ended_keys_to_extract = ['duration', 'timestamp', 'src_ip']

src_ips = set()
src_ports = set()
dst_ports = set()
dst_ips = set()

# map containing ip as key and timestamp as value 
ip_timestamps = {}

for entry in data:
    if entry['eventid'] == "cowrie.login.failed":
        new_entry = dict([(key, entry[key]) for key in login_keys_to_extract])
        login_attempts.append(new_entry)

    if entry['eventid'] == 'cowrie.session.connect': 
        new_entry = dict([(key, entry[key]) for key in session_keys_to_extract])
        session_details.append(new_entry)
        
        src_ports.add(entry['src_port'])
        dst_ports.add(entry['dst_port'])
        dst_ips.add(entry['dst_ip'])
    
    if entry['eventid'] == 'cowrie.login.success':
        new_entry = dict([(key, entry[key]) for key in login_keys_to_extract])
        login_attempts.append(new_entry)
     
    if entry['eventid'] == 'cowrie.session.closed':
        new_entry = dict([(key, entry[key]) for key in session_ended_keys_to_extract])
        session_ended_details.append(new_entry)
    
    if entry['eventid'] == 'cowrie.command.input':
        commands.append(entry['input'])

    src_ips.add(entry['src_ip'])
    timestamps.append(entry['timestamp'])
    ip_timestamps[entry['src_ip']] = entry['timestamp']


print(f"Source - IPs: {src_ips}, Ports: {src_ports}")
print(f"Destination - IPs: {dst_ips}, Ports: {dst_ports}")
print(f"Commands: {commands}")
[print(entry) for entry in login_attempts]
print(f"IPs with timestamps: {ip_timestamps}")

#print(session_details)



Source - IPs: {'172.19.0.1'}, Ports: {60874, 47574, 41478}
Destination - IPs: {'172.19.0.2'}, Ports: {2222}
Commands: ['ls', 'whoami', 'wget https://google.com']
{'username': 'root', 'password': 'abc', 'timestamp': '2023-03-02T14:28:05.022834Z', 'src_ip': '172.19.0.1', 'session': '6bba26d07dad', 'message': 'login attempt [root/abc] succeeded'}
{'username': 'admin', 'password': 'jlasjdf', 'timestamp': '2023-03-02T14:28:32.829234Z', 'src_ip': '172.19.0.1', 'session': '7ea852079e2b', 'message': 'login attempt [admin/jlasjdf] failed'}
{'username': 'admin', 'password': 'asdf', 'timestamp': '2023-03-02T14:28:36.334548Z', 'src_ip': '172.19.0.1', 'session': '7ea852079e2b', 'message': 'login attempt [admin/asdf] failed'}
{'username': 'admin', 'password': 'admin', 'timestamp': '2023-03-02T14:28:39.058641Z', 'src_ip': '172.19.0.1', 'session': '7ea852079e2b', 'message': 'login attempt [admin/admin] failed'}
{'username': 'root', 'password': 'abc', 'timestamp': '2023-03-02T14:29:46.727211Z', 'src_ip

Tag rules when created 

Save rules to a .txt or .csv file 

Would be convenient if we could create rules with their id within python and then send it to the firewall. Store id somewhere

Delete firewall rules based on timestamps? 

ufw 

Python function to get all rules to create and all rules to delete 
- create: all incoming connections from the last 24 hours
- delete: all the ones that has not been seen during the last 24 hours 